# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Y. Wang  ->  Y. Wang  |  ['Y. Wang']
K. Paterson  ->  K. Paterson  |  ['K. Paterson']
M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
M. Flock  ->  M. Flock  |  ['M. Flock']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']


Arxiv has 75 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.10829
extracting tarball to tmp_2507.10829...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.11072
extracting tarball to tmp_2507.11072...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2507.11072/Paterson_ghosts.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authors' from 'tmp_2507.11072/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'acronyms' from 'tmp_2507.11072/acronyms.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

Found 77 bibliographic references in tmp_2507.11156/FukuharaFlock25.bbl.
Retrieving document from  https://arxiv.org/e-print/2507.11326


extracting tarball to tmp_2507.11326... done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2507.11326/paper.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'app_orth_poly.tex' from 'tmp_2507.11326/app_orth_poly.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'app_properties.tex' from 'tmp_2507.11326/app_properties.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9

/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/nl_tv1.pdf 
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\linewidth]{figures/nl_tv1.pdf }
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 47 bibliographic references in tmp_2507.11326/paper.bbl.
Issues with the citations
syntax error in line 165: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.11072-b31b1b.svg)](https://arxiv.org/abs/2507.11072) | **Euclid VI. NISP-P optical ghosts**  |
|| E. Collaboration, et al. -- incl., <mark>K. Paterson</mark>, <mark>M. Schirmer</mark>, <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-07-16*|
|*Comments*| *13 pages, 10 figures, submitted*|
|**Abstract**|            The Near-Infrared Spectrometer and Photometer (NISP) onboard Euclid includes several optical elements in its path, which introduce artefacts into the data from non-nominal light paths. To ensure uncontaminated source photometry, these artefacts must be accurately accounted for. This paper focuses on two specific optical features in NISP's photometric data (NISP-P): ghosts caused by the telescope's dichroic beamsplitter, and the bandpass filters within the NISP fore-optics. Both ghost types exhibit a characteristic morphology and are offset from the originating stars. The offsets are well modelled using 2D polynomials, with only stars brighter than approximately 10 magnitudes in each filter producing significant ghost contributions. The masking radii for these ghosts depend on both the source-star brightness and the filter wavelength, ranging from 20 to 40 pixels. We present the final relations and models used in the near-infrared (NIR) data pipeline to mask these ghosts for Euclid's Quick Data Release (Q1).         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.11156-b31b1b.svg)](https://arxiv.org/abs/2507.11156) | **Hydrodynamical simulations of the vertical shear instability with dynamic dust and cooling rates in protoplanetary disks**  |
|| Y. Fukuhara, <mark>M. Flock</mark>, S. Okuzumi, R. T. Tominaga |
|*Appeared on*| *2025-07-16*|
|*Comments*| *16 pages, 20 figures, 1 table, Accepted for publication in Astronomy & Astrophysics (A&A)*|
|**Abstract**|            Turbulence in protoplanetary disks affects dust evolution and planetesimal formation. The vertical shear instability (VSI) is one of the candidate turbulence-driving mechanisms in the outer disk region. Since the VSI requires rapid gas cooling, dust grains in disks can influence and potentially control VSI-driven turbulence. However, VSI-driven turbulence has strong vertical motion, causing vertical dust diffusion. As a result, it remains unclear how turbulent structures and dust distributions form. We aim to clarify whether the VSI can achieve a quasi-steady dust profile under cooling rate evolution associated with turbulently diffusing dust. We also elucidate the dependence of the dust size and dust-to-gas mass ratio on the realization and persistence of the equilibrium state. We perform global two-dimensional hydrodynamical simulations of an axisymmetric disk to investigate how the VSI drives turbulence and maintains a balance between dust settling and diffusion. These simulations account for the dynamic interplay between dust distribution, cooling rates, and turbulence. We find that VSI mixing, dust settling, and local cooling reach an equilibrium, forming a thick dust layer with a dimensionless vertical mixing coefficient of approximately 10^{-3}. The ability of the VSI to sustain this state also depends on the dust size and dust-to-gas mass ratio. Larger grains or lower mass ratios weaken turbulence, leading to dust settling. The condition of equilibrium state existence is consistent with the prediction of the semi-analytic model presented by Fukuhara & Okuzumi (2024). Our results indicate that efficient turbulent dust mixing and efficient cooling can occur simultaneously. They also imply that turbulence in VSI-dominated disks has different intensity levels depending on the grain size. This suggests that the efficiency of dust growth can depend on the VSI in protoplanetary disks.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.11326-b31b1b.svg)](https://arxiv.org/abs/2507.11326) | **Euclid preparation. Overview of Euclid infrared detector performance from ground tests**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-07-16*|
|*Comments*| *22 pages, 20 figures, 4 pages of annexes. Submitted to A&A*|
|**Abstract**|            The paper describes the objectives, design and findings of the pre-launch ground characterisation campaigns of the Euclid infrared detectors. The pixel properties, including baseline, bad pixels, quantum efficiency, inter pixel capacitance, quantum efficiency, dark current, readout noise, conversion gain, response nonlinearity, and image persistence were measured and characterised for each pixel. We describe in detail the test flow definition that allows us to derive the pixel properties and we present the data acquisition and data quality check software implemented for this purpose. We also outline the measurement protocols of all the pixel properties presented and we provide a comprehensive overview of the performance of the Euclid infrared detectors as derived after tuning the operating parameters of the detectors. The main conclusion of this work is that the performance of the infrared detectors Euclid meets the requirements. Pixels classified as non-functioning accounted for less than 0.2% of all science pixels. IPC coupling is minimal and crosstalk between adjacent pixels is less than 1% between adjacent pixels. 95% of the pixels show a QE greater than 80% across the entire spectral range of the Euclid mission. The conversion gain is approximately 0.52 ADU/e-, with a variation less than 1% between channels of the same detector. The reset noise is approximately equal to 23 ADU after reference pixels correction. The readout noise of a single frame is approximately 13 $e^-$ while the signal estimator noise is measured at 7 $e^-$ in photometric mode and 9 $e^-$ in spectroscopic acquisition mode. The deviation from linear response at signal levels up to 80 k$e^-$ is less than 5% for 95% of the pixels. Median persistence amplitudes are less than 0.3% of the signal, though persistence exhibits significant spatial variation and differences between detectors.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.10829-b31b1b.svg)](https://arxiv.org/abs/2507.10829) | **VegasAfterglow: A High-Performance Framework for Gamma-Ray Burst Afterglows**  |
|| <mark>Y. Wang</mark>, C. Chen, B. Zhang |
|*Appeared on*| *2025-07-16*|
|*Comments*| *Submitted to Journal of High Energy Astrophysics. Comments are welcome. Code available at this https URL*|
|**Abstract**|            Gamma-ray bursts (GRBs) are the most luminous astrophysical transients, known to be associated with core collapse of massive stars or mergers of two compact objects such as two neutron stars. They are followed by multi-wavelength afterglow emission originating from the deceleration of the relativistic jets by the ambient medium. The study of after emission offers crucial insights into the physics of relativistic shocks, the properties of the circumburst environment, as well as the physical and geometrical structure of relativistic jets, as well as the viewing geometry of the observer. We present VegasAfterglow, a newly developed, high-performance C++ framework designed for modeling GRB afterglows with flexibility and computational efficiency as keynotes of design. The framework self-consistently solves forward and reverse shock dynamics and calculates synchrotron (including self-absorption or all spectral regimes) and inverse Compton radiation (including Klein-Nishina corrections); it can handle arbitrary user-defined ambient density profiles, central engine activity histories, viewing angles, and the jet structures of energy, Lorentz factor, and magnetization profiles. It supports both relativistic and non-relativistic regimes and includes lateral jet spreading effects. In this paper, we describe the numerical implementation of the framework and assess its computational performance. Our results demonstrate that VegasAfterglow is well-suited for interpreting current and future multi-wavelength observations in the era of multi-messenger astronomy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.11072/./figures/dichroic_wave.png', 'tmp_2507.11072/./figures/filter_wave.png', 'tmp_2507.11072/./figures/dichroic_radius_mag.png', 'tmp_2507.11072/./figures/filter_cen_radius_mag.png', 'tmp_2507.11072/./figures/dichroic_flux.png', 'tmp_2507.11072/./figures/filter_flux.png']
copying  tmp_2507.11072/./figures/dichroic_wave.png to _build/html/
copying  tmp_2507.11072/./figures/filter_wave.png to _build/html/
copying  tmp_2507.11072/./figures/dichroic_radius_mag.png to _build/html/
copying  tmp_2507.11072/./figures/filter_cen_radius_mag.png to _build/html/
copying  tmp_2507.11072/./figures/dichroic_flux.png to _build/html/
copying  tmp_2507.11072/./figures/filter_flux.png to _build/html/
exported in  _build/html/2507.11072.md
    + _build/html/tmp_2507.11072/./figures/dichroic_wave.png
    + _build/html/tmp_2507.11072/./figures/filter_wave.png
    + _build/html/tmp_2507.11072/./figures/dichroic_radius_mag.png
    + _build/html/tmp_2507.11072/./figures/filter_cen_r

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# Euclid: VI. NISP-P optical ghosts

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.11072-b31b1b.svg)](https://arxiv.org/abs/2507.11072)<mark>Appeared on: 2025-07-16</mark> -  _13 pages, 10 figures, submitted_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Paterson</mark>, <mark>M. Schirmer</mark>, <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The Near-Infrared Spectrometer and Photometer (NISP) onboard $\Euclid$ includes several optical elements in its path, which introduce artefacts into the data from non-nominal light paths. To ensure uncontaminated source photometry, these artefacts must be accurately accounted for. This paper focuses on two specific optical features in NISP's photometric data (NISP-P): ghosts caused by the telescope's dichroic beamsplitter, and the bandpass filters within the NISP fore-optics. Both ghost types  exhibit a characteristic morphology and are offset from the originating stars. The offsets are well modelled using 2D polynomials, with only stars brighter than approximately 10 magnitudes in each filter producing significant ghost contributions. The masking radii for these ghosts depend on both the source-star brightness and the filter wavelength, ranging from 20 to 40 pixels. We present the final relations and models used in the near-infrared (NIR) data pipeline to mask these ghosts for $\Euclid$ 's Quick Data Release (Q1).

</div>

<div id="div_fig1">

<img src="tmp_2507.11072/./figures/dichroic_wave.png" alt="Fig3.1" width="50%"/><img src="tmp_2507.11072/./figures/filter_wave.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** Shape of the dichroic ghost (_top row_) and filter ghost (_lower rows_) as a function of waveband. For the dichroic ghosts, images are made from a median combination of 60 $\times$ 60 pixel (18 $\times$ 18$\arcsec$) cutouts, restricted to one region of the FPA and a single magnitude bin. For the filter ghosts, since there is a dependency of the shape on the FPA position, images are made from a median combination of 80 $\times$ 80 pixel (24 $\times$ 24$\arcsec$) cutouts, restricted to either the central region (middle) or a corner region (bottom) of the FPA and a single magnitude bin. The number of cutouts used to create each median is given in the top right of each median cutout. (*fig:wave*)

</div>
<div id="div_fig2">

<img src="tmp_2507.11072/./figures/dichroic_radius_mag.png" alt="Fig8.1" width="50%"/><img src="tmp_2507.11072/./figures/filter_cen_radius_mag.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** The radius-magnitude relation, given by the coefficients in \cref{table:rad}, for each filter for the dichroic ghost (_left_) and filter ghost (_right_); described in \cref{sc:Rad_di}. (*fig:rad_mag*)

</div>
<div id="div_fig3">

<img src="tmp_2507.11072/./figures/dichroic_flux.png" alt="Fig2.1" width="50%"/><img src="tmp_2507.11072/./figures/filter_flux.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Appearance of the dichroic ghost (_top row_) and filter ghost (_bottom row_) as a function of source-star brightness. For the dichroic ghosts, images are made from a median combination of 60 $\times$ 60 pixel (18 $\times$ 18$\arcsec$) cutouts, restricted to one region of the FPA for the \HE band. For the filter ghosts, images are made from a median combination of 80 $\times$ 80 pixel (24 $\times$ 24$\arcsec$) cutouts, restricted to the central region of the FPA for the \YE band. The number of cutouts used to create each median is given in the top right of each median cutout. (*fig:flux*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.11072"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Hydrodynamical simulations of the vertical shear instability with dynamic dust and cooling rates in protoplanetary disks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.11156-b31b1b.svg)](https://arxiv.org/abs/2507.11156)<mark>Appeared on: 2025-07-16</mark> -  _16 pages, 20 figures, 1 table, Accepted for publication in Astronomy & Astrophysics (A&A)_

</div>
<div id="authors">

Y. Fukuhara, <mark>M. Flock</mark>, S. Okuzumi, R. T. Tominaga

</div>
<div id="abstract">

**Abstract:** Turbulence in protoplanetary disks affects dust evolution and planetesimal formation.The vertical shear instability (VSI) is one of the candidate turbulence-driving mechanisms in the outer disk region.Since the VSI requires rapid gas cooling, dust particles in disks can influence and potentially control VSI-driven turbulence.However, VSI-driven turbulence has strong vertical motion, causing vertical diffusion of particles.As a result of this interaction, it remains unclear how turbulent structures and dust distributions form and persist. We aim to clarify whether the VSI can drive turbulence and achieve a quasi-steady dust distribution under cooling rate evolution associated with turbulently diffusing dust.We also elucidate the dependence of the dust size and dust-to-gas mass ratio on the realization and persistence of the equilibrium state. We perform global two-dimensional hydrodynamical simulations of an axisymmetric protoplanetary disk to investigate how the VSI drives turbulence and maintains a balance between dust settling and diffusion.These simulations account for the dynamic interplay between dust distribution, cooling rates, and turbulence. We find that VSI mixing, dust settling, and local dust cooling reach an equilibrium, forming a thick dust layer with a dimensionless vertical mixing coefficient of approximately $\alpha_{\rm diff} \approx 10^{-3}$ .The ability of the VSI to sustain this state also depends on the dust size and dust-to-gas mass ratio.Larger grains or lower mass ratios weaken turbulence, leading to dust settling.The condition of equilibrium state existence is consistent with the prediction of the semi-analytic model presented by [ and Fukuhara (2024)]() . Our results indicate that efficient turbulent dust mixing and efficient cooling can occur simultaneously.They also imply that turbulence in VSI-dominated disks has different levels of intensity depending on the grain size.This suggests that the efficiency of dust growth can depend on the VSI in protoplanetary disks.

</div>

<div id="div_fig1">

<img src="tmp_2507.11156/./colormap_vz_rhodgINI_D2G1e-2_a1e-3_h.png" alt="Fig15" width="100%"/>

**Figure 15. -** Gas vertical velocity $v_{z,\rm gas}/c_{\rm s}$(left) and ratio of dust and gas density $(\rho_{\rm dust}/\rho_{\rm gas})/Z_{\rm D2G}$(right) for a run with $(Z_{\rm D2G}, a) = (0.01, 10 {\rm \mu m})$ in the dynamic-dust simulations, as a function of $R$ and $z$ at the end of the simulation ($1000$ orbits at $20 {\rm au}$). (*fig:colormap_vz_rhodgINI_D2G1e-2_a1e-3_h*)

</div>
<div id="div_fig2">

<img src="tmp_2507.11156/./colormap_time_vz_rhodgINI_D2G1e-2_a1e-3_h.png" alt="Fig16" width="100%"/>

**Figure 16. -** Gas vertical velocity $v_{z,\rm gas}/c_{\rm s}$(left) and ratio of dust and gas density $(\rho_{\rm dust}/\rho_{\rm gas})/Z_{\rm D2G}$(right) for a run with $(Z_{\rm D2G}, a) = (0.01, 10 {\rm \mu m})$ in the dynamic-dust simulations, as a function of time and $z/H_{\rm gas}$ at $R=40 {\rm au}$. (*fig:colormap_time_vzINI_rhodg_D2G1e-2_a1e-3_h*)

</div>
<div id="div_fig3">

<img src="tmp_2507.11156/./vertical_profile_rhod.png" alt="Fig2" width="100%"/>

**Figure 2. -** Vertical profiles of the dust density $\rho_{\rm dust}$ normalized by the initial dust density at the midplane $\rho_{\rm dust,ini}(z=0)$(upper panel), and ratio of dust and gas densities $\rho_{\rm dust}/\rho_{\rm gas}$ normalized by the initial dust-to-gas mass ratio $Z_{\rm D2G}$(lower panel), at $R=40 {\rm au}$ for a run with $(Z_{\rm D2G}, a) = (0.01, 10 {\rm \mu m})$ in the dynamic-dust simulations. In the upper panel, the dashed curve shows a Gaussian fitting of $\rho_{\rm dust}$ within $|z|<H_{\rm dg}$, and the vertical dotted lines mark the height of the dust scale height $H_{\rm dust}$. The horizontal dashed and dotted lines in the lower panel mark $\rho_{\rm dust}/\rho_{\rm gas}=Z_{\rm D2G}/2$ and $Z_{\rm D2G}$, respectively. The gray regions in both panels map the dust layer defined by $H_{\rm dg}$. (*fig:vertical_profile_rhod*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.11156"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\WG}[1]{\textcolor{green}{#1}}$
$\newcommand{\as}[1]{\textcolor{cyan}{#1}}$
$\newcommand{\Ha}{\ensuremath{\text{H}\alpha} }$
$\newcommand{\TBD}[1]{\textcolor{red}{\bf #1}}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$ preparation: TBD. Overview of $\Euclid$ infrared detector performance from ground tests

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.11326-b31b1b.svg)](https://arxiv.org/abs/2507.11326)<mark>Appeared on: 2025-07-16</mark> -  _22 pages, 20 figures, 4 pages of annexes. Submitted to A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The paper describes the objectives, design and findings of the pre-launch ground characterisation campaigns of the $\Euclid$ infrared detectors. The aim of the ground characterisations is to evaluate the performance of the detectors, to calibrate the pixel response, and to derive the pixel response correction methods. The detectors have been tested and characterised in the facilities set up for this purpose. The pixel properties, including baseline, bad pixels, quantum efficiency, inter pixel capacitance, quantum efficiency, dark current, readout noise, conversion gain, response nonlinearity, and image persistence were measured and characterised for each pixel. We describe in detail the test flow definition that allows us to derive the pixel properties and we present the data acquisition and data quality check software implemented for this purpose. We also outline the measurement protocols of all the pixel properties presented and we provide a comprehensive overview of the performance of the $\Euclid$ infrared detectors as derived after tuning the operating parameters of the detectors.The main conclusion of this work is that the performance of the infrared detectors $\Euclid$ meets the requirements. Pixels classified as non-functioning accounted for less than 0.2 \% of all science pixels. IPC coupling is minimal and crosstalk between adjacent pixels is less than 1 \% between adjacent pixels. 95 \% of the pixels show a QE greater than 80 \% across the entire spectral range of the Euclid mission. The conversion gain is approximately 0.52 ADU/e $^-$ , with a variation less than 1 \% between channels of the same detector. The reset noise is approximately equal to 23 ADU after reference pixels correction. The readout noise of a single frame is approximately $\SI{13}{e^-}$ while the signal estimator noise is measured at 7 e $^-$ in photometric mode and 9 e $^-$ in spectroscopic acquisition mode. The deviation from linear response at signal levels up to 80 ke $^-$ is less than 5 \% for 95 \% of the pixels. Median persistence amplitudes are less than 0.3 \% of the signal, though persistence exhibits significant spatial variation and differences between detectors.

</div>

<div id="div_fig1">

<img src="tmp_2507.11326/./figures/pers_nonsat_TV1.png" alt="Fig17" width="100%"/>

**Figure 17. -** Persistence amplitudes in percentage of the previous flat-field illumination below saturation. (*fig:pers_det_structures*)

</div>
<div id="div_fig2">

<img src="tmp_2507.11326/./figures/H2RG_geom.png" alt="Fig3" width="100%"/>

**Figure 3. -**  H2RG frame geometry definition with reference pixels on the edges of the array in the 32-output channel mode. The read directions are indicated by the blue arrows. The widths are not in scale.  (*fig:frame_geometry*)

</div>
<div id="div_fig3">

<img src="tmp_2507.11326/./figures/nl_ref_lin.png" alt="Fig15" width="100%"/>

**Figure 15. -**  Example of linear flux definition using ramps acquired in UTR(394). Fit performed using 76 frames $S_\textrm{lin}^{(76)}$ and 394 frames $S_\textrm{lin}^{(394)}$. The difference per pixel of first order coefficients was computed and the relative differences between the median signals per detector are shown for two detectors: low persistence SCA18452 (blue points) a higher persistence SCA18267 (red squares). (*fig:nl_ref_lin*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.11326"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

143  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

17  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
